In [ ]:
try:
    from threadpoolctl import threadpool_limits
    threadpool = True
except ImportError:
    threadpool = False
from nestmodel.centralities import calc_katz_iter, calc_katz
from convergence_helper import MultiTracker, ParameterWrapper, save_results
from convergence_helper import get_datasets, get_samples

In [ ]:
datasets = ["karate",
            "phonecalls",
            "HepPh",
            "AstroPh",
#            "web-Google",
#            "soc-Pokec"
           ]

In [ ]:
from convergence_helper import process_dataset
names_to_eigenvalues = {
    "karate":      6.725697727631732,
    "phonecalls" : 13.125956990198913,
    "HepPh":       20, #5.6629203197371965,
    "AstroPh":     94.4415437598996,
    "web-Google":  37.639582115220364,
    "soc-Pokec":   118.52742163417824}

def compute_on_all_datasets(datasets, tracker, params):
    """ computes
    """

    for dataset in datasets:
        alpha=1/3 * 1/names_to_eigenvalues[dataset]
        params.cent_kwargs["alpha"]=alpha
        process_dataset(dataset, tracker, params)

In [ ]:
tracker = MultiTracker(("katz",))
params = ParameterWrapper(
dataset_path = None,
cent_func    = calc_katz_iter,
cent_kwargs  = {"epsilon":1e-15,
                "max_iter":100},
rewire_kwargs =    dict(method=1, source_only=True),
wl_kwargs = {"initial_colors": None, "both":False},
number_of_samples=get_samples(100)
)

params.verbosity=1
params.tqdm=1
tracker.verbosity=1
if threadpool:
    with threadpool_limits(limits=1, user_api='blas'):
        with threadpool_limits(limits=1, user_api='openmp'):
            compute_on_all_datasets(get_datasets(datasets), tracker, params)
else:
    compute_on_all_datasets(get_datasets(datasets), tracker, params)

In [ ]:
save_results("katz", tracker, params)